# SISTER workflow

In [18]:
import json
import os
import xml.etree.ElementTree as ET
import pandas as pd
import IPython
from itertools import groupby

# Import warnings module and ignore warnings in output below
import warnings
warnings.filterwarnings("ignore")

# Import and initialize MAAP class
from maap.maap import MAAP
maap = MAAP(maap_host="sister-api.imgspec.org")

### Create unique scene identifier

In [15]:
granules = ['https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20200911170127_20200911170131_0001.zip',
            'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20220802165941_20220802165946_0001.zip']

meta = 'EXAMPLE_RUN'

scenes = []

crid = "990"

for l1_granule in granules:
    
    landsat = 'None'   

    base_name = os.path.basename(l1_granule)

    if base_name.startswith('DESIS'):
        sensor = 'DESIS'
        datetime = base_name[31:46]

    elif base_name.startswith('PRS'):
        sensor = 'PRISMA'
        datetime = base_name[16:24] + 'T' + base_name[24:30]
        landsat='https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_%s_landsat.tar.gz' % base_name[16:50]

    elif base_name.startswith('ang'):
        sensor = 'AVNG'
        datetime = base_name[3:18]

    elif base_name.startswith('f'):
        sensor = 'AVCL'
        ''' AVIRIS classic filenames do not contain acquisition times,to be consistent with other
            sensors and to ensure identifier codes are unique a time string is created using other
            numbers in the filename            
        '''     

        datetime = "20%sT%s%s%s" % (base_name[1:7],
                                    base_name[8:10],
                                    base_name[11:13],
                                    base_name[14:16])
    else:
        raise ValueError('Unrecognized L1 datafile')

    job_args = {'sensor': sensor,
                'datetime': datetime,
                 'crid' : crid}
    job_args['preprocess'] = {'raw_dataset': l1_granule,
                              'landsat_dataset' : landsat}

    print(job_args)
    
    scenes.append(job_args)


{'sensor': 'PRISMA', 'datetime': '20200911T170127', 'crid': '990', 'preprocess': {'raw_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20200911170127_20200911170131_0001.zip', 'landsat_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_20200911170127_20200911170131_0001_landsat.tar.gz'}}
{'sensor': 'PRISMA', 'datetime': '20220802T165941', 'crid': '990', 'preprocess': {'raw_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/raw/PRS_L1_STD_OFFL_20220802165941_20220802165946_0001.zip', 'landsat_dataset': 'https://sister-ops-workspace.s3.us-west-2.amazonaws.com/prisma/landsat_reference/PRS_20220802165941_20220802165946_0001_landsat.tar.gz'}}


## Step 1. Preprocess

In [16]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        queue="sister-job_worker-32gb"
    else:
        queue="sister-job_worker-16gb"
    
    identifier = f'SISTER_{scene["sensor"]}_L1B_RDN_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_job_response = maap.submitJob(
        algo_id = "sister-preprocess",
        version = "sister-dev",
        raw_dataset = scene['preprocess']['raw_dataset'],
        landsat_dataset = scene['preprocess']['landsat_dataset'],
        crid = scene['crid'],
        publish_to_cmr = False,
        cmr_metadata={},
        queue=queue,
        identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {preprocess_job_response.status}')
    print(f'Job ID: {preprocess_job_response.id}')
          
    scenes[i]['preprocess']['job_id'] = preprocess_job_response.id
    

Identifier: SISTER_PRISMA_L1B_RDN_20200911T170127_990_EXAMPLE_RUN
Submission status: success
Job ID: 87de9507-fda5-4af1-9528-10789591abea
Identifier: SISTER_PRISMA_L1B_RDN_20220802T165941_990_EXAMPLE_RUN
Submission status: success
Job ID: 6e6785dc-1e01-4a55-a9f7-0bf056aced1c


## Step 2. ISOFIT


In [17]:
for i,scene in enumerate(scenes):
        
    if scene['sensor'] == 'AVCL':
        segmentation_size = 100
    else:
        segmentation_size = 100
    
    identifier = f'SISTER_{scene["sensor"]}_L2A_RFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    preprocess_id = scene['preprocess']['job_id'] 
    
    preprocess_result= [ x for x in maap.getJobResult(preprocess_id).outputs if x.startswith("s3://s3.") and "RDN" in x]
    preprocess_result.sort()
    l1b_rdn,l1b_loc, l1b_obs = preprocess_result
    
    scene['preprocess']['radiance_dataset'] =l1b_rdn
    scene['preprocess']['location_dataset'] =l1b_loc
    scene['preprocess']['observation_dataset'] =l1b_obs

    isofit_job_response = maap.submitJob(
                                    algo_id="sister-isofit",
                                    version="sister-dev",
                                    radiance_dataset=l1b_rdn,
                                    location_dataset = l1b_loc,
                                    observation_dataset = l1b_obs,
                                    segmentation_size = segmentation_size,
                                    n_cores=32,
                                    crid = scene['crid'],
                                    publish_to_cmr=False,
                                    cmr_metadata={},
                                    queue="sister-job_worker-32gb",
                                    identifier= identifier)

    print(f'Identifier: {identifier}')
    print(f'Submission status: {isofit_job_response.status}')
    print(f'Job ID: {isofit_job_response.id}')
    
    scene['isofit']  = {'job_id' : isofit_job_response.id}


Identifier: SISTER_PRISMA_L2A_RFL_20200911T170127_990_EXAMPLE_RUN
Submission status: success
Job ID: fb24d389-4a58-4879-81a2-3851ca91556a
Identifier: SISTER_PRISMA_L2A_RFL_20220802T165941_990_EXAMPLE_RUN
Submission status: success
Job ID: 6c250617-0c68-4d6c-8e52-b9203c750b98


## Step 3. Spectral resample

In [19]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_RSRFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    isofit_id = scene['isofit']['job_id'] 

    iso_result= [ x for x in maap.getJobResult(isofit_id).outputs if x.startswith("s3://s3.") and "RFL" in x]
    l2a_rfl,l2a_unc = iso_result
        
    scene['isofit']['reflectance_dataset'] =l2a_rfl
    scene['isofit']['uncertainty_dataset'] =l2a_unc

    resample_job_response = maap.submitJob(
                                            algo_id="sister-resample",
                                            version="sister-dev",
                                            reflectance_dataset= l2a_rfl,
                                            uncertainty_dataset= l2a_unc,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % resample_job_response.status)
    print('Job ID: %s' % resample_job_response.id)
    scene['resample']  = {'job_id' : resample_job_response.id}


Identifier: SISTER_PRISMA_L2A_RSRFL_20200911T170127_990_EXAMPLE_RUN
Submission status: success
Job ID: 7cba3815-997e-4044-822e-264977ad6b34
Identifier: SISTER_PRISMA_L2A_RSRFL_20220802T165941_990_EXAMPLE_RUN
Submission status: success
Job ID: 790dd272-3449-4f0d-88a4-8a3c8eef3d42


## Step 3. Reflectance correction

In [20]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2A_CORFL_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    resample_id = scene['resample']['job_id'] 
    resample_result= [ x for x in maap.getJobResult(resample_id).outputs if x.startswith("s3://s3.") and "RSRFL" in x]
    l2a_rsrfl,l2a_rsunc = resample_result
    
    scene['resample']['reflectance_dataset'] =l2a_rsrfl
    scene['resample']['uncertainty_dataset'] =l2a_rsunc



    rfl_corr_job_response = maap.submitJob(
                                            algo_id="sister-reflect_correct",
                                            version="sister-dev",
                                            observation_dataset= scene['preprocess']['observation_dataset'],
                                            reflectance_dataset= l2a_rsrfl,
                                            crid = scene['crid'],
                                            publish_to_cmr=False,
                                            cmr_metadata={},
                                            queue="sister-job_worker-16gb",
                                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % rfl_corr_job_response.status)
    print('Job ID: %s' % rfl_corr_job_response.id)
    scene['reflect_correct']  = {'job_id' : rfl_corr_job_response.id}


Identifier: SISTER_PRISMA_L2A_CORFL_20200911T170127_990_EXAMPLE_RUN
Submission status: success
Job ID: 4351f071-ecc9-4c24-a43d-abac921cb564
Identifier: SISTER_PRISMA_L2A_CORFL_20220802T165941_990_EXAMPLE_RUN
Submission status: success
Job ID: 51200a63-4619-4821-a009-6c333a0f5a72


## Step 4. Fractional Cover

In [21]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_FRCOV_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    correct_id = scene['reflect_correct']['job_id'] 
    correct_result= [ x for x in maap.getJobResult(correct_id).outputs if x.startswith("s3://s3.") and "CORFL" in x]
    l2a_corfl =  correct_result[0]
    scene['reflect_correct']['reflectance_dataset'] = l2a_corfl

    frcover_job_response = maap.submitJob(
                                        algo_id="sister-fractional-cover",
                                        version="sister-dev",
                                        reflectance_dataset=l2a_corfl,
                                        n_cores= 20,
                                        refl_scale= 1,
                                        normalization = 'brightness',
                                        crid = scene['crid'],
                                        publish_to_cmr=False,
                                        cmr_metadata={},
                                        queue="sister-job_worker-32gb",
                                        identifier= identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % frcover_job_response.status)
    print('Job ID: %s' % frcover_job_response.id)
    scene['frcover']  = {'job_id' : frcover_job_response.id}

Identifier: SISTER_PRISMA_L2B_FRCOV_20200911T170127_990_EXAMPLE_RUN
Submission status: success
Job ID: 52246a88-c613-4a69-838a-d14fff3f0788
Identifier: SISTER_PRISMA_L2B_FRCOV_20220802T165941_990_EXAMPLE_RUN
Submission status: success
Job ID: e86a43b5-5db7-4063-86d8-2476b131408f


## Step 6a. Vegetation biochemistry

In [22]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_VEGBIOCHEM_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    vegbiochem_job_response = maap.submitJob(
                            algo_id="sister-trait_estimate",
                            version="sister-dev",
                            reflectance_dataset= scene['reflect_correct']['reflectance_dataset'],
                            frcov_dataset=l2b_frcov,
                            veg_cover = 0.5,
                            crid = scene['crid'],
                            publish_to_cmr=False,
                            cmr_metadata={},
                            queue="sister-job_worker-16gb",
                            identifier=identifier)

    print(f'Identifier: {identifier}')
    print('Submission status: %s' % vegbiochem_job_response.status)
    print('Job ID: %s' % vegbiochem_job_response.id)
    scene['vegbiochem']  = {'job_id' : vegbiochem_job_response.id}

Identifier: SISTER_PRISMA_L2B_VEGBIOCHEM_20200911T170127_990_EXAMPLE_RUN
Submission status: success
Job ID: dd64be4f-85d1-4a07-9b07-96c1e7709d40
Identifier: SISTER_PRISMA_L2B_VEGBIOCHEM_20220802T165941_990_EXAMPLE_RUN
Submission status: success
Job ID: 16d626eb-284a-401b-a296-d39915372ce3


## Step 6b. Snow grainsize

In [ ]:
for i,scene in enumerate(scenes):

    identifier = f'SISTER_{scene["sensor"]}_L2B_GRAINSIZE_{scene["datetime"]}_{scene["crid"]}_{meta}'
    
    frcover_id = scene['frcover']['job_id'] 
    frcover_result= [x for x in maap.getJobResult(frcover_id).outputs if x.startswith("s3://s3.") and "FRCOV" in x]
    l2b_frcov =  frcover_result[0]                   
    scene['frcover']['frcover_dataset'] = l2b_frcov

    grainsize_job_response = maap.submitJob(
        algo_id="sister-grainsize",
        version="sister-dev",
        reflectance_dataset=  scene['reflect_correct']['reflectance_dataset'],
        frcov_dataset= scene['frcover']['frcover_dataset'],
        snow_cover = 0.9,
        crid= scene['crid'],
        publish_to_cmr=False,
        cmr_metadata={},
        queue="sister-job_worker-16gb",
        identifier=identifier)
    
    print(f'Identifier: {identifier}')
    print('Submission status: %s' % grainsize_job_response.status)
    print('Job ID: %s' % grainsize_job_response.id)
    scene['grainsize']  = {'job_id' : grainsize_job_response.id}

In [12]:
vegbiochem_job_response = maap.submitJob(
algo_id="sister-trait_estimate",
version="sister-dev",
reflectance_dataset='s3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987',
frcov_dataset='s3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2B_FRCOV/2018/05/14/SISTER_AVNG_L2B_FRCOV_20180514T042654_987',
veg_cover = 0.5,
crid = '987',
publish_to_cmr=False,
cmr_metadata={},
queue="sister-job_worker-16gb",
identifier='SISTER_AVNG_L2B_VEGBIOCHEM_20180514T042654_987')

In [11]:
maap.getJobResult('0eff29c9-2350-40da-be87-e7e58c3f9fd2').outputs


['http://sister-ops-workspace.s3-website.us-west-2.amazonaws.com/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987',
 's3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987',
 'https://s3.console.aws.amazon.com/s3/buckets/sister-ops-workspace/LOM/PRODUCTS/AVNG/L2A_CORFL/2018/05/14/SISTER_AVNG_L2A_CORFL_20180514T042654_987/?region=us-east-1&tab=overview',
 'http://sister-ops-workspace.s3-website.us-west-2.amazonaws.com/anonymous/dps_output/sister-reflect_correct/sister-dev/2023/02/08/19/38/36/071295',
 's3://s3.us-west-2.amazonaws.com:80/sister-ops-workspace/anonymous/dps_output/sister-reflect_correct/sister-dev/2023/02/08/19/38/36/071295',
 'https://s3.console.aws.amazon.com/s3/buckets/sister-ops-workspace/anonymous/dps_output/sister-reflect_correct/sister-dev/2023/02/08/19/38/36/071295/?region=us-east-1&tab=overview']